In [4]:
import os
import numpy
import iris
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot
import cartopy
import cartopy.crs
import bokeh
import bokeh.layouts
import datetime
import bokeh.models
import bokeh.plotting
from bokeh.io import show, output_notebook, push_notebook
output_notebook()

ModuleNotFoundError: No module named 'numpy'

In [5]:
def get_radar_colours():

    """Return dictionary of precip. colormap and normalisation."""

    radar_colours1 = [(220 / 255.0, 220 / 255.0, 220 / 255.0, 1.0),
                      (122 / 255.0, 147 / 255.0, 212 / 255.0, 0.9),
                      (82 / 255.0, 147 / 255.0, 212 / 255.0, 0.95),
                      (39 / 255.0, 106 / 255.0, 188 / 255.0, 1.0),
                      (31 / 555.0, 201 / 255.0, 26 / 255.0, 1.0),
                      (253 / 255.0, 237 / 255.0, 57 / 255.0, 1.0),
                      (245 / 255.0, 152 / 255.0, 0 / 255.0, 1.0),
                      (235 / 255.0, 47 / 255.0, 26 / 255.0, 1.0),
                      (254 / 255.0, 92 / 255.0, 252 / 255.0, 1.0),
                      (255 / 255.0, 255 / 255.0, 255 / 255.0, 1.0)]

    radar_levels = numpy.array(
        [0.0, 0.1, 0.25, 0.5, 1.0, 2.0, 4.0, 8.0, 16.0, 32.0])
    cmap_radar, norm_radar = matplotlib.colors.from_levels_and_colors(
        radar_levels, radar_colours1, extend='max')
    cmap_radar.set_name = 'radar'
    
    return {'cmap': cmap_radar, 'norm': norm_radar}

In [6]:
def get_image_array_from_figure(fig):
    """
    Get the RGB buffer from the matplotlib figure.
    """
    fig.canvas.draw()
    h, w = fig.canvas.get_width_height()
    # print(' width={0}\n height={1}'.format(w, h))

    buf = numpy.fromstring(fig.canvas.tostring_argb(), dtype=numpy.uint8)
    # print('buf shape', buf.shape)
    buf.shape = (w, h, 4)

    # canvas.tostring_argb give pixmap in ARGB mode. Roll the ALPHA channel to
    # have it in RGBA mode
    buf = numpy.roll(buf, 3, axis=2)
    buf = numpy.flip(buf, axis=0)
    return buf

In [7]:
# Load data
SEA_REGION_DICT = {'indonesia': [-15.1, 1.0865, 99.875, 120.111],
                   'malaysia': [-2.75, 10.7365, 95.25, 108.737],
                   'phillipines': [3.1375, 21.349, 115.8, 131.987],
                   'se_asia': [-18.0, 29.96, 90.0, 153.96],
                   }
data_bounds = SEA_REGION_DICT['se_asia']

# create file paths
print('create file paths')
base_dir = '/s3/stephen-sea-public-london/model_data'
model_run_str = '20171208T1200'
path_list1 = [os.path.join(base_dir,f1)
               for f1 in os.listdir(base_dir)
               if model_run_str in f1]

print('load cube for a particular variable and time from files: %s' % path_list1)
stash_section1 = 4
stash_item1 = 203
selected_time = datetime.datetime(2017,12,9,10,30,0)
cube_func1 = lambda cube1: \
    cube1.attributes['STASH'].section == \
    stash_section1 and \
    cube1.attributes['STASH'].item == \
    stash_item1 and \
    len(cube1.cell_methods) > 0

cell_func_dict1 = {
    'time': lambda cell1: abs(cell1.point - selected_time) < datetime.timedelta(seconds=1)}


ic1 = iris.Constraint(cube_func=cube_func1)
# ,
#                       coord_values=cell_func_dict1)
data_cube = iris.load_cube(path_list1[0], ic1)
# data_cube.convert_units('kg-m-2-hour^-1')
data_cube = data_cube * 60*60
data_cube

create file paths


FileNotFoundError: [Errno 2] No such file or directory: '/s3/stephen-sea-public-london/model_data'

In [8]:
example_cube = data_cube[0]
example_cube.has_lazy_data()

NameError: name 'data_cube' is not defined

In [9]:
def create_plot(cube, index):
    print('create matplotlib plot')
    plot_options = {}
    plot_options['precipitation'] = get_radar_colours()
    figure_name = 'test01'
    current_figure = matplotlib.pyplot.figure(figure_name, figsize=(8.0, 6.0))
    current_axes = \
        current_figure.add_subplot(
            111,
            projection=cartopy.crs.PlateCarree())

    current_axes.coastlines(resolution='110m')
    coords_lat = example_cube.coords('latitude')[0].points
    coords_long = example_cube.coords('longitude')[0].points
    current_var = 'precipitation'

    main_plot = current_axes.pcolormesh(coords_long,
                                     coords_lat,
                                     example_cube.data,
                                     cmap=plot_options[current_var]['cmap'],
                                     norm=plot_options[current_var]['norm'],
                                     transform=cartopy.crs.PlateCarree()
                                       )

    current_title = 'test_plot'

    current_axes.set_position([0, 0, 1, 1])
    current_axes.set_xlim(data_bounds[2], data_bounds[3])
    current_axes.set_ylim(data_bounds[0], data_bounds[1])
    current_axes.xaxis.set_visible(False)
    current_axes.yaxis.set_visible(False)
    return current_figure
    
current_figure = create_plot(data_cube, 0)

NameError: name 'data_cube' is not defined

In [10]:

img_arr1 = get_image_array_from_figure(current_figure)

cur_region = data_bounds
x_limits = bokeh.models.Range1d(cur_region[2], cur_region[3],
                                bounds=(cur_region[2], cur_region[3]))
y_limits = bokeh.models.Range1d(cur_region[0], cur_region[1],
                                bounds=(cur_region[0], cur_region[1]))


bokeh_figure = bokeh.plotting.figure(plot_width=800,
                                     plot_height=600,
                                     x_range=x_limits,
                                     y_range=y_limits,
                                     tools='pan,wheel_zoom,reset,save')

latitude_range = cur_region[1] - cur_region[0]
longitude_range = cur_region[3] - cur_region[2]
dummy_img1 = numpy.zeros((300,400,4),dtype='uint8')
# pdb.set_trace()
bokeh_image = bokeh_figure.image_rgba(image=[img_arr1],
                                      x=[cur_region[2]],
                                      y=[cur_region[0]],
                                      dw=[longitude_range],
                                      dh=[latitude_range])

mid_x = (cur_region[2] + cur_region[3]) * 0.5
mid_y = (cur_region[0] + cur_region[1]) * 0.5

bokeh_img_ds = bokeh_image.data_source

bokeh_figure.title.text = 'Title...'
main_layout = bokeh.layouts.column(bokeh_figure)



update_handel = show(main_layout, notebook_handle=True)

NameError: name 'current_figure' is not defined

In [11]:
from ipywidgets import widgets
from IPython.display import display
times = {str(datetime.datetime.fromtimestamp(p * 60 * 60)) :  i for i,p in enumerate(data_cube.coord('time').points)}
dropdown = widgets.Dropdown(options=times)
x = []
def ishow(change):
    if change['type'] == 'change' and change['name'] == 'value':
        index = change['new']
        img = get_image_array_from_figure(create_plot(data_cube, index))
        data = bokeh_img_ds.data
        bokeh_img_ds.data = dic(
            x=data['x'],
            y=data['y'],
            dw=data['dw'],
            dh=data['dh'],
            image=[img])
        print('new image')
        push_notebook(handle=update_handel)
    
dropdown.observe(ishow)
display(dropdown)

NameError: name 'data_cube' is not defined